In [ ]:
from github import Github
import pandas as pd 
import itertools
import json
import os


#### Enter personal token here to authenticate.

In [ ]:
%store -r access_token
g = Github(access_token)

In [ ]:
g.rate_limiting

#### These variables are global variables

In [ ]:
repo_var = None
page_num=None
repo_list=list()
list_without_lesson=list()
list_with_lesson=list()
final_repo_list=list()

#### Declaring a function . 
* To get the total repositories count
* Creating a page_num variable to hold the valid page numbers to iterate the repo list.
* Final repo list created from repo list
* List with lesson created from final repo list
* List without lesson created from final repo list

In [ ]:
def repo_initialize():
    
    '''
    Function takes no paramter.
    Returning total repositories count, final repository list, list_with_lesson & list_without_lesson.
    
    '''
    
    
    global repo_var
    global page_num
    global repo_list
    global list_without_lesson
    global list_with_lesson
    global final_repo_list
    
    repo_var = g.get_organization("carpentries-incubator").get_repos()

    
# Creating a page_num variable to hold the valid page numbers to iterate the repo list

    page_num = 0
    for i in range(0,10):
        if len(repo_var.get_page(i))==0:
            page_num = i
            break
        else:
            pass
    repo_list = list()
    for i in range(0,page_num):
        repo_list.append(repo_var.get_page(i))
    final_repo_list = list((itertools.chain.from_iterable(repo_list)))

# This program gives two Repository lists. 1)list_with_lesson & list_without_lesson.

    for repo in final_repo_list:
        tags=repo.get_topics()
        if "lesson" in tags:
            list_with_lesson.append(repo)
        else:
            list_without_lesson.append(repo)
            
# Calling the function

repo_initialize()
    

#### KPI 1 > Declaring a functions to get the commit  , Pulls , issues counts and contributors and life -cycle stage of each repositories.

In [ ]:
#### DataFrame columns headings

cols=["repo","commits","open_pulls","open_issues","commit_contributors", "issue_contributors","pulls_contributors"]
df=pd.DataFrame(columns=cols)

### Declaring a function to get repositroies

def get_repolist(list_with_lesson):
    
    '''
    Functions taking list_with_lesson as a parameter
    Returning repolist in df
    '''
    repos=[]
    for repo in list_with_lesson:
        if  os.path.exists('data/'+repo.name+'.json'):
            repos.append(repo.name)
    return repos

        
#### Declaring a function to get commit count and commit authors of each repositories.
    
def get_commitcount_commitauthors(list_with_lesson):
    
    '''
    Functions taking list_with_lesson as a parameter
    Returning  commitlist & commit_authorlist
    
    '''
    commitlist=[] # holds the total count of number of commits made in the repositories.
    commit_authorlist=[] # contains authors of each commit.
    for repo in list_with_lesson:
        commit_count=0
        commit_author_count=0
        author_list=[]
        if not os.path.exists('data/'+repo.name+'.json'):
            continue
        with open('data/'+repo.name+'.json','r',encoding='utf-8') as f:
            fildata=json.load(f)
            for k in fildata['commits_dict'].keys():
                c=fildata['commits_dict'][k]
                commit_count+=1
                author_list.append(c[0]) 
            commitlist.append(commit_count)
            commit_authorlist.append(len(set(author_list)))
    return commitlist,commit_authorlist


#### Declaring a function to get pullrequest count and contributor of each repositories.
        
def get_prcount_prauthorcount(list_with_lesson):
    pr_list=[]
    pr_authorlist=[]
    for repo in list_with_lesson:
        pr_count=0
        pr_author_count=0
        author_list=[]
        if not os.path.exists('data/'+repo.name+'.json'):
            continue
        with open('data/'+repo.name+'.json','r') as f:
            try:
                fildata=json.load(f)

                for k in fildata['pr_dict']:
                    pr_count+=1
                    author_list.append(k['pr_name'])
            except:
                pass
        pr_list.append(pr_count)
        pr_authorlist.append(len(set(author_list)))
    return pr_list,pr_authorlist



#### Declaring a function to get issue count and contributor of each repositories.

def get_issuecount_issue_user_count(list_with_lesson):
    issue_list=[]
    issue_authorlist=[]
    for repo in list_with_lesson:
        if not os.path.exists('data/'+repo.name+'.json'):
            continue
        issue_count=0
        issue_author_count=0
        author_list=[]
        with open('data/'+repo.name+'.json','r',encoding='utf-8') as f:
            fildata=json.load(f)
            
            for k in fildata['issue_dict']:
                issue_count+=1
                author_list.append(k['user_name'])
        issue_list.append(issue_count)
        issue_authorlist.append(len(set(author_list)))
    return issue_list,issue_authorlist



#### Declaring a function to get life cycle stage of each repositories.

def get_stages(list_with_lesson):
    stage_list=[]
    for repo in list_with_lesson:
        if not os.path.exists('data/'+repo.name+'.json'):
            continue
#         print(repo.name)
        with open('data/'+repo.name+'.json','r',encoding='utf-8') as f:
            fildata=json.load(f)
            stage_list.append(fildata['stage'])
    return stage_list

#### Calling get_commitcount_commitauthors(list_with_lesson)
        
commitlist,commit_authorlist=get_commitcount_commitauthors(list_with_lesson)

#### Calling get_prcount_prauthorcount(list_with_lesson)

pr_list,pr_authorlist=get_prcount_prauthorcount(list_with_lesson)

#### Calling get_issuecount_issue_user_count(list_with_lesson)

issue_list,issue_authorlist=get_issuecount_issue_user_count(list_with_lesson)

#### Calling get_stages(list_with_lesson)

stage_list=get_stages(list_with_lesson)

df['repo']=get_repolist(list_with_lesson)
df['stage']=stage_list
df['commits']=commitlist
df['commit_contributors']=commit_authorlist
df['open_pulls']=pr_list
df['pulls_contributors']=pr_authorlist 
df['open_issues']=issue_list
df['issue_contributors']=issue_authorlist



#### KPI 2 > Plotting dataframe to analyse the contrinutors of each paramter  of 111 repositories of Carpentry Incubator.

In [ ]:
pd.set_option('display.max_rows', None)
df


#### KPI 3 > Function to calculate total_controbutors.

In [ ]:
def calculate_total_contributors(df):

    df["Total Contributor"]=df['commit_contributors']+df['issue_contributors']+df['pulls_contributors']

calculate_total_contributors(df)

In [ ]:
pd.set_option('display.max_rows', None)
df


#### KPI 3 > To get average number of contributors for a lesson ( classifying by different stages: pre-alpha, alpha, beta, stable)


In [ ]:
df[['commit_contributors','issue_contributors','pulls_contributors','stage']].groupby(['stage']).mean()

#### KPI 4> To get average  of total contributors of a lesson ( classifying by different stages:pre-alpha, alpha, beta,stable)

In [ ]:
df[['stage', 'Total Contributor']].groupby(['stage']).mean()